In [ ]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [ ]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# 
# использую это во 2 блоке, поэтому может необязательно для скачивания
#!pip install pandas-profiling

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB,MultinomialNB
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler , Normalizer , PowerTransformer , QuantileTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,HalvingGridSearchCV, KFold 
from sklearn.model_selection import RandomizedSearchCV, ShuffleSplit, StratifiedShuffleSplit , LeaveOneOut
from sklearn.metrics import r2_score , accuracy_score, precision_score, recall_score ,log_loss,balanced_accuracy_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import RocCurveDisplay, DetCurveDisplay
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import validation_curve
from IPython.display import display_html

### Область работы 2 (поиск лучшей модели)

In [ ]:
# Данный блок(и) НЕ выполняются преподавателем в области 2
# блок(и) предназначены для поиска лучшей модели
# 
# Запрещается размещать данные блоки за пределами обасти 2
# Все блоки данной области должны быть выполнены
#
# Путь к тренировочному набору
# 

path_train = 'wine_train.csv' # содержит только имя файла, без имен папок !
df_raw = pd.read_csv(path_train)

# меняем качества вин, где 7 и более -  1, обратное - 0
df_raw['quality'] = np.where(df_raw['quality'] >= 7 , 1 , 0)
display(df_raw)

# ПОЧИСТИМ НАШИ ДАННЫЕ

#нулевые значения в данных(проверял на всех и нашел только в этом столбце)
#убеждаемся, что есть дубликаты и нужно будет их отчистить

#df_row.duplicated().sum() дубликатов - 560--- нужно их удалить

temp = df_raw.drop_duplicates(keep = 'last',ignore_index= True) # сначала дропаем дубликаты
oshibki = temp[temp['citric acid'] == 0.000].index # находим индексы где у нас есть 0 , можно будет убрать их
# подготовленный фрейм для работы
df = temp.drop(index = oshibki).reset_index(drop = True)#[coll]
display(df)


In [ ]:
# можно посмотреть на наши данные 
for i in df.columns:
    sns.histplot(df_raw[i])
    plt.show()


In [ ]:
# проверить, что нет дубликатов
df.duplicated().sum()

In [ ]:
# сводка данных по фреймам
#otchet = df_raw.profile_report()
otchet1 = df.profile_report()
display(otchet1)

In [ ]:
sns.pairplot(df, hue='quality')

In [ ]:
#можно глянуть на тепловую карту
correlation_matrix = df.corr()
# Выводим признаки на тепловую карту
plt.figure(figsize= (10, 6))
sns.heatmap(correlation_matrix, annot = True)

In [ ]:
# блок для простого анализа
X_train, X_test, y_train, y_test = train_test_split(df.drop(['quality'],axis = 1), df['quality'], test_size =0.3, random_state=42)

scaler = RobustScaler() 
scaler.fit(X_train)     

X_train_scaled = scaler.transform(X_train) 
X_test_scaled = scaler.transform(X_test)   


knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(X_train_scaled, y_train)

sc_train = knn.score(X_train_scaled , y_train)
sc_test =  knn.score(X_test_scaled ,  y_test)

print("тренировочный: {:.3f}".format(sc_train))
print("тестовый: {:.3f}".format(sc_test))

## KNN

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['quality'],axis = 1), df['quality'], test_size =0.3, random_state=42)


In [ ]:
pipe = Pipeline([
    ('preprocessing', StandardScaler()), 
    ('regressor',    KNeighborsClassifier())])


#cv = KFold()
#cv = ShuffleSplit(n_splits = 10,shuffle=True)
cv = KFold(n_splits= 15, shuffle= True) 
#cv = StratifiedShuffleSplit(n_splits = 15)
#cv = LeaveOneOut()
#cv = StratifiedKFold(n_splits=5, shuffle=True)

# Определим параметры для перебора по решетке
p = np.arange(1,6)            
n_neighbors = np.arange(1,16)
weights = ['uniform','distance']
scaling = [ MinMaxScaler(), StandardScaler(),RobustScaler(),Normalizer()]

param_grid = [ 
    {'preprocessing':scaling,
     'regressor': [KNeighborsClassifier()],
     'regressor__p':p,
     'regressor__n_neighbors': n_neighbors,
     'regressor__weights': weights}]

grid = GridSearchCV(pipe, param_grid, cv=cv, return_train_score=True, n_jobs=-1, scoring= 'accuracy')
grid.fit(X_train,y_train)
grid

In [ ]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

In [ ]:
print("----------------- Обучили и тестировали -------------------")
print("Наилучшие параметры:\n{}\n".format(grid.best_params_))
print("Средняя правильность для наилучшей модели кроссвалидации на" 
      "валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
print("Правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid.score(X_test, y_test)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['quality'],axis = 1), df['quality'], test_size =.25, random_state = 42)


finish_pipe1 =  Pipeline([
            ('preprocessing', RobustScaler()), 
            ('regressor',     KNeighborsClassifier(n_neighbors = 13, p = 4, weights = 'distance'))
            ])
finish_pipe1.fit(X_train, y_train)

print(finish_pipe1.score(X_train,y_train))
print(finish_pipe1.score(X_test,y_test))

## Log regression 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['quality'],axis = 1), df['quality'], test_size =.3, random_state = 42)



In [ ]:
pipe = Pipeline([('preprocessing', RobustScaler()), 
                 ('clf',           LogisticRegression(max_iter=3000))])

# n_splits = 5, random_state = 42 дают не улачную комбинацию и это хорошо для разбора ошибок решетки

#cv = StratifiedShuffleSplit(n_splits = 10)
#cv = ShuffleSplit(n_splits = 25)
#cv = LeaveOneOut()
cv = KFold(n_splits= 20, shuffle= True)

scaling = [ MinMaxScaler(), StandardScaler(),RobustScaler(), Normalizer(), PowerTransformer(), QuantileTransformer()]

param_grid =[
    {'preprocessing': scaling,'clf__penalty': ['l2'], 
         'clf__solver': ['newton-cg' ,'lbfgs', 'liblinear', 'sag', 'saga']},
    {'preprocessing': scaling,'clf__penalty': ['l1'], 
         'clf__solver': ['liblinear']},
    {'preprocessing': scaling,'clf__penalty': ['none'], 
         'clf__solver': ['lbfgs','newton-cg']}
    ]

grid = GridSearchCV(pipe, param_grid, cv = cv, return_train_score = True, scoring= 'accuracy')
grid.fit(X_train, y_train)
grid 

In [ ]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

In [ ]:
print("Наилучшие параметры:\n{}\n".format(grid.best_params_))
print("Средняя правильность для наилучшей модели кроссвалидации на" 
      "валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
print("Правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid.score(X_test, y_test)))

In [ ]:
# Подставляем параметры в контейнер
X_train, X_test, y_train, y_test = train_test_split(df.drop(['quality'],axis = 1), df['quality'], test_size =.3, random_state = 42)

finish_pipe2 =  make_pipeline(QuantileTransformer(), LogisticRegression(solver = 'liblinear', penalty = 'l2', max_iter=4500) )
finish_pipe2.fit(X_train, y_train)

print(finish_pipe2.score(X_train,y_train))
print(finish_pipe2.score(X_test,y_test))

In [ ]:
# Ну тут можно подкрутить "С", но если penalty != none
# можно подкрутить модель
pipe_fin = make_pipeline(QuantileTransformer(), LogisticRegression(solver = 'liblinear', penalty = 'l2', max_iter=4500))


n_range = np.linspace(0.0001, 6, 500)
train_scores, test_scores = validation_curve(
    pipe_fin , X_train, y_train,
    param_name = "logisticregression__C", 
    param_range = n_range,
    cv = cv, 
    scoring= "accuracy", 
    n_jobs=-1)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure(figsize=(15,10))

plt.title("Validation Curve")
plt.xlabel("сила регуляризации")
plt.ylabel("Score")
plt.ylim(0.7, 1.05)

plt.plot(n_range, train_scores_mean, label="Training score", color="darkorange")
plt.fill_between(n_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange")
plt.plot(n_range, test_scores_mean, label="Cross-validation score",
             color="navy")
plt.fill_between(n_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy")

plt.xticks(np.linspace(0, 6, 15))
plt.legend(loc="best")
plt.show()

In [ ]:
pipe_end = make_pipeline(QuantileTransformer(), LogisticRegression(solver = 'liblinear', penalty = 'l2', max_iter=4500, C=0.4))
pipe_end.fit(X_train,y_train)
print(pipe_end.score(X_train,y_train))
print(pipe_end.score(X_test,y_test))

## Гаус

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['quality'],axis = 1), df['quality'], test_size =.3, random_state = 42)

In [ ]:
pipe = Pipeline([('preprocessing', RobustScaler()), 
                 ('clf',           GaussianNB())])

#cv = StratifiedShuffleSplit(n_splits = 20)
#cv = ShuffleSplit(n_splits = 20)
cv = KFold(n_splits= 20, shuffle= True)

scaling = [ MinMaxScaler(), StandardScaler(), RobustScaler(), Normalizer(), PowerTransformer(), QuantileTransformer()]

param_grid =[
    {'preprocessing': scaling, 
     'clf': [GaussianNB()]}
    ]

grid = GridSearchCV(pipe, param_grid, cv = cv, return_train_score = True, scoring= 'accuracy')
grid.fit(X_train, y_train)
grid 

In [ ]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

In [ ]:
print("Наилучшие параметры:\n{}\n".format(grid.best_params_))
print("Средняя правильность для наилучшей модели кроссвалидации на" 
      "валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
print("Правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid.score(X_test, y_test)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['quality'],axis = 1), df['quality'], test_size =0.25, random_state=42)

finish_pipe3 = Pipeline(steps=[('preprocessing', PowerTransformer()), ('clf', GaussianNB())])
finish_pipe3.fit(X_train, y_train)

print(finish_pipe3.score(X_train,y_train))
print(finish_pipe3.score(X_test,y_test))

## Выбор модели

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['quality'],axis = 1), df['quality'], test_size =0.3, random_state = 42)


model_NB = make_pipeline(PowerTransformer(),GaussianNB())
model_LR = make_pipeline(QuantileTransformer(), LogisticRegression(solver = 'liblinear', penalty = 'l2', max_iter=4500, C=0.4))
model_KNN = make_pipeline(RobustScaler(), KNeighborsClassifier(n_neighbors = 13, p = 4, weights = 'distance'))


model_NB.fit(X_train,y_train)
model_LR.fit(X_train,y_train)
model_KNN.fit(X_train,y_train)

y_NB_pred = model_NB.predict(X_test)
y_LR_pred = model_LR.predict(X_test)
y_KNN_pred = model_KNN.predict(X_test)

In [ ]:

df1 = pd.DataFrame(classification_report(y_test, y_NB_pred, output_dict=True)).T
df2 = pd.DataFrame(classification_report(y_test, y_LR_pred, output_dict=True)).T
df3 = pd.DataFrame(classification_report(y_test, y_KNN_pred, output_dict=True)).T

df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption('GaussianNB')
df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption('LogisticRegression')
df3_styler = df3.style.set_table_attributes("style='display:inline'").set_caption('KNN')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_()+df3_styler._repr_html_(), raw=True)

In [ ]:
finish_pipe1.fit(X_train,y_train)
finish_pipe2.fit(X_train,y_train)
finish_pipe3.fit(X_train,y_train)

y_pred_1 = finish_pipe1.predict(X_test)
y_pred_2 = finish_pipe2.predict(X_test)
y_pred_3 = finish_pipe3.predict(X_test)



matrix_1=confusion_matrix(y_test,y_pred_1)
matrix_2=confusion_matrix(y_test,y_pred_2)
matrix_3=confusion_matrix(y_test,y_pred_2)


fig,ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(15,5))
ax = plt.subplot(1, 3, 1)
sns.heatmap(matrix_1,annot=True, fmt='g', cbar=None,cmap="Blues");
ax = plt.subplot(1, 3, 2)
sns.heatmap(matrix_2,annot=True, fmt='g', cbar=None,cmap="Blues");
ax = plt.subplot(1, 3, 3)
sns.heatmap(matrix_3,annot=True, fmt='g', cbar=None,cmap="Blues");

In [ ]:
fig, ax_roc = plt.subplots(1,1, figsize=(10, 5))

RocCurveDisplay.from_estimator(finish_pipe1, X_test, y_test, ax = ax_roc, name = 'pipe_1');
RocCurveDisplay.from_estimator(finish_pipe2, X_test, y_test, ax = ax_roc, name = 'pipe_2');
RocCurveDisplay.from_estimator(finish_pipe3, X_test, y_test, ax = ax_roc, name = 'pipe_3');

In [ ]:
fig, [ax_roc, ax_det] = plt.subplots(1,2, figsize=(10, 5))

RocCurveDisplay.from_estimator(finish_pipe1, X_test, y_test, ax = ax_roc, name = 'pipe_1');
RocCurveDisplay.from_estimator(finish_pipe2, X_test, y_test, ax = ax_roc, name = 'pipe_2');
RocCurveDisplay.from_estimator(finish_pipe3, X_test, y_test, ax = ax_roc, name = 'pipe_3');

DetCurveDisplay.from_estimator(finish_pipe1, X_test, y_test, ax = ax_det, name = 'pipe_1');
DetCurveDisplay.from_estimator(finish_pipe2, X_test, y_test, ax = ax_det, name = 'pipe_2');
DetCurveDisplay.from_estimator(finish_pipe3, X_test, y_test, ax = ax_det, name = 'pipe_3');

### Область работы 3 (выполнение лучшей модели)

In [ ]:
# Данный блок(и) в области 3 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [ ]:
# Путь к тренировочному набору
path_train = 'wine_train.csv' # содержит только имя файла, без имен папок
df_raw = pd.read_csv(path_train)
df_raw['quality'] = np.where(df_raw['quality'] >= 7 , 1 , 0)
temp = df_raw.drop_duplicates(keep = 'last',ignore_index= True) # сначала дропаем дубликаты
# 0 значения есть только здесь(заранее заметил это)
oshibki = temp[temp['citric acid'] == 0.000].index # находим индексы где у нас есть 0 , можно будет убрать их
df = temp.drop(index = oshibki).reset_index(drop = True)
#display(df)

# Путь к тестовому набору
path_test  = 'wine_test.csv' # содержит только имя файла, без имен папок
df1_raw = pd.read_csv(path_test)
ind_duble = df1_raw[df1_raw.duplicated(keep = 'last') == True].index
df2_raw = df1_raw.drop(index = ind_duble).reset_index(drop = True)
oshibki1 = df2_raw[df2_raw['citric acid'] == 0.000].index # ошибка была тут-я индексы ошибок брал с df1_raw
df1 = df2_raw.drop(index = oshibki1).reset_index(drop = True) # а нужно было с обрезанного, а не изначального
display(df1)                                                  

#я тут неправильно переобозначил фреймы, если посмотреть изначальный файл - теперь исправил
# нужно было всего написать цифру или по другому фрейм назвать в 3 местах.

In [ ]:
# Блок(и) обучения и поверки модели

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['quality'],axis = 1), df['quality'], test_size =.3)


finish_pipe =  Pipeline([
            ('preprocessing', RobustScaler()), 
            ('regressor',     KNeighborsClassifier(n_neighbors = 13, p = 3, weights = 'distance'))
            ])
finish_pipe.fit(X_train, y_train)

print(finish_pipe.score(X_train,y_train))
print(finish_pipe.score(X_test,y_test))

In [ ]:
# Блок предсказания с использованием тестового набора

In [ ]:
y_predict = finish_pipe.predict(df1)
y_predict.shape

In [ ]:
# Название вектора предсказанных значений  y_predict полученого на основании тестового набора
y_predict = y_predict 

## Подстановка

In [ ]:
column = ['quality']
# вставить файл с данными в ''
y_true_raw = pd.read_csv('.....').[column] 
y_true_raw['quality'] = np.where(y_true_raw['quality'] >= 7 , 1 , 0)
# Срезаем фрейм по удаленным индексам : ind_duble, oshibki1
temp1 = y_true_raw.drop(index = ind_duble).reset_index(drop = True)
y_true = temp1.drop(index = oshibki1).reset_index(drop = True)
# готовый 
y_true

#ошибка в оформелнии к тестовому набору 